In [2]:
from azureml.core import Workspace
from azureml.core.model import Model

In [4]:
# Load ID
import json

azure_id = open('my_id.json', 'r')
mi = json.load(azure_id)
my_id = mi['my_id']

In [ ]:
ws = Workspace.create(name="actividad2",
                      subscription_id = my_id,
                      resource_group = "actividad2",
                      location = "centralindia")

In [8]:
mname = "model"
registered_model = Model.register(model_path="reglog.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [17]:
from azureml.core.environment import Environment
virtual_env = Environment("env-1-bankruptcy")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])

In [21]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\soalv\AppData\Local\Temp\ipykernel_30804\480079058.py:9: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [22]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [23]:
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()